In [177]:
import pathlib
import sys 
import random
import json
import cv2 
import base64
import requests as req
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [195]:
#constants
PART_NAMES = [
    "nose", "leftEye", "rightEye", "leftEar", "rightEar", "leftShoulder",
    "rightShoulder", "leftElbow", "rightElbow", "leftWrist", "rightWrist",
    "leftHip", "rightHip", "leftKnee", "rightKnee", "leftAnkle", "rightAnkle"
]
PART_IDS = {pn: pid for pid, pn in enumerate(PART_NAMES)} ;PART_IDS
CONNECTED_PART_NAMES = [
    ("leftHip", "leftShoulder"), ("leftElbow", "leftShoulder"),
    ("leftElbow", "leftWrist"), ("leftHip", "leftKnee"),
    ("leftKnee", "leftAnkle"), ("rightHip", "rightShoulder"),
    ("rightElbow", "rightShoulder"), ("rightElbow", "rightWrist"),
    ("rightHip", "rightKnee"), ("rightKnee", "rightAnkle"),
    ("leftShoulder", "rightShoulder"), ("leftHip", "rightHip")
]

CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES]; CONNECTED_PART_INDICES

[(11, 5),
 (7, 5),
 (7, 9),
 (11, 13),
 (13, 15),
 (12, 6),
 (8, 6),
 (8, 10),
 (12, 14),
 (14, 16),
 (5, 6),
 (11, 12)]

In [32]:
def getBase64Image(filePath):
    with open(filePath, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
        my_string = my_string.decode('utf-8')
    return my_string
    

In [238]:
def get_adjacent_keypoints(keypoints):
    results = []
    for left, right in CONNECTED_PART_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results
            
        

In [222]:
my_string = getBase64Image('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/Screenshot 2020-01-28 at 7.10.58 AM.png')
 

In [170]:
#Loading model
r = req.get(url='http://localhost:3000/loadModel')


In [172]:
#Sending data to server
url2 = 'http://localhost:3000/postImage'
data = {'imgBase64':'data:image/png;base64,'+my_string}
r = req.post(url=url2 , data = data)

In [173]:
poses = r.json()
len(poses['detectionList'])
len(poses['detectionList'])

10

In [175]:
poses

{'detectionList': [{'score': 0.9562930710175458,
   'keypoints': [{'score': 0.9639641046524048,
     'part': 'nose',
     'position': {'x': 1124.2798657006538, 'y': 200.98228668794678}},
    {'score': 0.9753755331039429,
     'part': 'leftEye',
     'position': {'x': 1128.373064128647, 'y': 197.53790574098582}},
    {'score': 0.9405955076217651,
     'part': 'rightEye',
     'position': {'x': 1122.3922066920704, 'y': 198.75073933394776}},
    {'score': 0.9766906499862671,
     'part': 'leftEar',
     'position': {'x': 1137.0908601453687, 'y': 200.7839154255039}},
    {'score': 0.5650030374526978,
     'part': 'rightEar',
     'position': {'x': 1121.1994922507122, 'y': 212.33357430826646}},
    {'score': 0.9988600015640259,
     'part': 'leftShoulder',
     'position': {'x': 1140.030384688788, 'y': 232.79873770155154}},
    {'score': 0.9987313747406006,
     'part': 'rightShoulder',
     'position': {'x': 1119.2667584573285, 'y': 229.7444651436764}},
    {'score': 0.9752274751663208,
  

In [232]:
def draw_keypoints(poses,img):
    cv_keypoints= []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. *score))
    out_img = cv2.drawKeypoints(img, cv_keypoints , outImage=np.array([]) ) 
    

In [230]:
def draw_skelton(poses,img):
    adjacent_keypoints = []
    out_img = img
    for pose in poses['detectionList']:
        keypoints  = pose['keypoints']
        new_keypoints = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoints)
    out_img = cv2.polylines(out_img , adjacent_keypoints, isClosed=False, color=(255,255,0))
    return out_img    

In [249]:
def draw_skel_and_kp(poses , img):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        new_keypoint = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoint)
        
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. * score))
    out_img = cv2.drawKeypoints(
        img, cv_keypoints , outImage=np.array([]), color=(255,0,0),
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS )
    out_img = cv2.polylines(out_img , adjacent_keypoints , isClosed=False, color=[255,255,0])
    return out_img
    
    
        

In [204]:
a =[[1,2][3,4]]
a[1,:,:]

TypeError: list indices must be integers or slices, not tuple

In [251]:
img = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/Screenshot 2020-01-28 at 7.10.58 AM.png')
out_img = draw_skel_and_kp(poses,img)
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/shootingPoset.jpg' ,out_img) 

True

In [187]:
out_img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [114]:
root_dir = '/Volumes/My Passport/FinalYearProjectData/ActionClassification2/train/3-Pointer'
filePaths = os.listdir(filePath)

In [118]:
for filename in filePaths:
    fileDir = root_dir+'/'+filename;
    img = cv2.imread(fileDir)
    
    url2 = 'http://localhost:3000/postImage'
    data = {'imgBase64':'data:image/png;base64,'+my_string}
    poses = req.post(url=url2 , data = data).json()
    
    draw_pose(poses , img)
    
    cv2.imshow('Pose detection',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
    
    
    

error: OpenCV(3.4.5) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:358: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [119]:
%debug

> <ipython-input-118-22f9a6c78c85>(11)<module>()
      9     draw_pose(poses , img)
     10 
---> 11     cv2.imshow('Pose detection',img)
     12     if cv2.waitKey(1) & 0xFF == ord('q'):
     13         break

ipdb> img
ipdb> img.size
*** AttributeError: 'NoneType' object has no attribute 'size'
ipdb> img.shape
*** AttributeError: 'NoneType' object has no attribute 'shape'
ipdb> pose
*** NameError: name 'pose' is not defined
ipdb> up
*** Oldest frame
ipdb> poses
{'detectionList': [{'score': 0.9250530951163348, 'keypoints': [{'score': 0.9930237531661987, 'part': 'nose', 'position': {'x': 295.6148355164881, 'y': 80.44114245775452}}, {'score': 0.974271297454834, 'part': 'leftEye', 'position': {'x': 305.41321453510096, 'y': 69.6828731872875}}, {'score': 0.9795336723327637, 'part': 'rightEye', 'position': {'x': 290.7708810887912, 'y': 70.44577619325312}}, {'score': 0.8615837097167969, 'part': 'leftEar', 'position': {'x': 319.79284176845033, 'y': 73.40683737443518}}, {'score': 0.75181901454

ipdb> exit


In [29]:
videoPath ='/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/PosenetUsingCDN.ipynb.mov'
cap = cv2.VideoCapture(videoPath)

count =0;
mergedImage = None;
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
       poses = 
    else:
        cap.release()
        break
cap.release()
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/mergedImage.png', mergedImage)   

True

In [15]:
apple1 = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/apple1.jpg')
apple2 = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/apple2.jpg')


apple1 = cv2.resize(apple1 , (450,450))
apple2 = cv2.resize(apple1 , (450,450))
change = apple1 + apple2 
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/change.jpg' , change)

True

In [136]:
img = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/Screenshot 2020-01-28 at 7.10.58 AM.png')

In [137]:
img= cv2.resize(img , (850,480))

In [138]:
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/smaller.png', img)

True